<head>
    <link rel="stylesheet" href="bathroom.css">
</head>

<div class="container">
    <div class="components">
        <!-- queue header and your info like eta -->
        <div class="div1">

            <!-- title -->
            <div class="queue-header">
                <h2 id="teacherName">Mr. Mortensen's Queue</h2>
            </div>

            <!-- where you are in the line and the ETA -->
            <div class="queue-info">
                <p id="position">Your position in line: #</p>
                <p id="estimatedTime">Estimated time: ~ mins</p>
            </div>

            <hr>
        </div>

        <!-- list of students in line -->
        <div class="div2">
            <div class="queue-list">
                <p>Students currently in line:</p>
                <ul id="studentList">
                    <!-- Students will be listed here -->
                </ul>

                <!-- different options and actions you can take -->
                <div class="queue-buttons queue-actions">
                    <button type="button" onclick="addToQueue()">Add Me to Queue</button>
                    <button type="button" onclick="removeFromQueue()">Remove Me from Queue</button>
                    <button type="button" onclick="goToProfile()">Go to Profile</button>
                    <button type="button" onclick="reportIssue()">Report Issue</button>
                    <button type="button" onclick="viewIssues()">View Issues</button>
                </div>
            </div>
        </div>
    </div>
</div>

<!-- imports for icons and other visual stuff -->
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<script type="module" src="https://unpkg.com/ionicons@7.1.0/dist/ionicons/ionicons.esm.js"></script>
<script nomodule src="https://unpkg.com/ionicons@7.1.0/dist/ionicons/ionicons.js"></script>

<script>
    // FUNCTIONS DEFINING WHAT THE BUTTONS DO
    function goToProfile() {
        // Redirect to profile page
        window.location.href = "{{site.baseurl}}/profile";
    }

    function reportIssue() {
        // Redirect to issue report page
        window.location.href = "{{site.baseurl}}/issue";
    }

    function viewIssues() {
        window.location.href = "{{site.baseurl}}/get-issues"
    }
    // Global variable to store queue data
    window.studentsInQueue = [];

    // show the queue and the students in it
    function displayQueue() {
        const studentList = document.getElementById("studentList"); // this is where teh students are displaed
        studentList.innerHTML = ""; // Clear the list

        // Display the queue by adding a new list element with the student's name
        window.studentsInQueue.forEach((student) => {
            const listItem = document.createElement("li");
            listItem.textContent = student;

            if (student === studentName) listItem.style.color = "red"; // Highlight your own name in red

            studentList.appendChild(listItem);
        });

        // Disable "Add Me to Queue" button if student is already in queue
        const addButton = document.querySelector('button[onclick="addToQueue()"]');
        addButton.disabled = window.studentsInQueue.includes(studentName);

        // Update position and estimated time
        const position = window.studentsInQueue.indexOf(studentName) + 1;
        document.getElementById("position").textContent = `Your position in line: #${position}`;
        document.getElementById("estimatedTime").textContent = `Estimated time: ~${position * 3} mins`;
    }

    // Initial display
    displayQueue();
</script>

<script>
    import { javaURI, fetchOptions } from '{{ site.baseurl }}/assets/js/api/config.js'

    const url = javaURI + '/api/queue';
    const getUrl = url + '/all';
    const addUrl = url + '/add';
    const removeUrl = url + '/remove';
    const approveUrl = url + '/approve';
    const namesUrl = `http://localhost:8085/mvc/person/read`;

    const fetchOptions = {
        headers: {
            "Content-Type": "application/json"
        }
    };

    const postOptions = {
        ...fetchOptions,
        method: 'POST',
    };
    const deleteOptions = {
        ...fetchOptions,
        method: 'DELETE',
    };

    // Get teacher name and student name from local storage or variables
    const teacherName = "Mortensen"; // Replace with dynamic teacher name if needed
    const studentName = localStorage.getItem("email"); // Assume stored from login

    fetch(namesUrl, fetchOptions)
        .then(response => {
            if (response.status !== 200) {
                console.error("Failed to fetch names.");
                return;
            }
            return response.json();
        })
        .then(data => {
            console.log("Names data:", data);
            const student = data.find(person => person.email === studentName);
            if (student) {
                console.log("Student name:", student.name);
                localStorage.setItem("name", student.name); // Store student name
            } else {
                console.error("Student not found.");
            }
        })
        .catch(error => console.error("Error fetching data:", error));

    function fetchQueueData() {
        fetch(getUrl, fetchOptions)
            .then(response => response.ok
                ? response.json() // response was ok; jsonify the response
                : Promise.reject("Failed to fetch queue data.") // response was not ok; error
            )
            .then(data => {
                console.log("Queue data:", data);

                // Find the queue for the current teacher
                const mortensenQueue = data.find(queue => queue.teacherName === teacherName);
                if (!mortensenQueue) {
                    console.error("Mortensen's queue not found.");
                    return;
                }

                // Update the global queue and display it
                window.studentsInQueue = mortensenQueue.peopleQueue.split(',');
                displayQueue();

                // Approve the first student if conditions are met
                if (window.studentsInQueue[0] === studentName) approveStudent();

                // Add "in bathroom" note for the first student if teacher is away
                if (mortensenQueue.away === 1 && window.studentsInQueue.length > 0) {
                    window.studentsInQueue[0] += " (in bathroom)";
                    displayQueue();
                }
            })
            .catch(error => console.error("Error fetching data:", error));
    }

    // Function to add student to the queue
    function addToQueue() {
        const queuer = {
            teacherName: teacherName,
            studentName: studentName,
        };

        fetch(addUrl, {
            ...postOptions,
            body: JSON.stringify(queuer),
        })
            .then(response => response.ok
                ? fetchQueueData() // response was ok: Refresh the queue after adding
                : alert("Failed to add to queue.") // response was not ok: display error
            )
            .catch(error => console.error("Error adding to queue:", error));
        console.log("Added to queue:", queuer);
    }

    // Function to remove student from the queue
    function removeFromQueue() {
        const queuer = {
            teacherName: teacherName,
            studentName: studentName,
        };

        fetch(removeUrl, {
            ...deleteOptions,
            body: JSON.stringify(queuer),
        })
            .then(response => response.ok
                ? fetchQueueData() // response was ok: Refresh the queue after removing
                : alert("Failed to remove from queue.") // response was not ok; display error
            )
            .catch(error => console.error("Error removing from queue:", error));
        console.log("Removed from queue:", queuer);
    }

    // Approves the first student in the queue and logs their check-in time
    function approveStudent() {
        // Log current time in HH:MM:SS format
        const now = new Date();
        const timeIn = `${now.getHours()}:${now.getMinutes().toString().padStart(2, "0")}:${now.getSeconds().toString().padStart(2, "0")}`;
        localStorage.setItem("timeIn", timeIn);

        // Prepare request payload with teacher's and student's names
        const queuer = {
            teacherName,
            studentName: window.studentsInQueue[0],
        };

        // Send approval request
        fetch(approveUrl, {
            ...postOptions,
            body: JSON.stringify(queuer),
        })
            .then(response => response.ok
                ? (window.location.href = "{{site.baseurl}}/hallpass") // response worked; redirect the user
                : alert("Failed to approve student.") // response failed; approval failed
            )
            .catch(error => console.error("Error approving student:", error));
    }

    // Automatically fetch queue data every 10 seconds
    setInterval(fetchQueueData, 10000);

    // Initial queue data fetch on page load
    fetchQueueData();

</script>